# Serialization
This notebook covers how to serialize chains to and from disk. The serialization format we use is json or yaml. Currently, only some chains support this type of serialization. We will grow the number of supported chains over time.


## Saving a chain to disk
First, let's go over how to save a chain to disk. This can be done with the `.save` method, and specifying a file path with a json or yaml extension.

In [2]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0), verbose=True)

In [3]:
llm_chain.save("llm_chain.json")

Let's now take a look at what's inside this saved file

In [4]:
!cat llm_chain.json

{
    "memory": null,
    "verbose": true,
    "tags": null,
    "metadata": null,
    "prompt": {
        "input_variables": [
            "question"
        ],
        "input_types": {},
        "output_parser": null,
        "partial_variables": {},
        "template": "Question: {question}\n\nAnswer: Let's think step by step.",
        "template_format": "f-string",
        "validate_template": false,
        "_type": "prompt"
    },
    "llm": {
        "_type": "giga-chat-model"
    },
    "output_key": "text",
    "output_parser": {
        "_type": "default"
    },
    "return_final_only": true,
    "llm_kwargs": {},
    "_type": "llm_chain"
}

## Loading a chain from disk
We can load a chain from disk by using the `load_chain` method.

In [4]:
from langchain.chains import load_chain

In [5]:
chain = load_chain("llm_chain.json")

In [6]:
chain.run("whats 2 + 2")



> Entering new LLMChain chain...
Prompt after formatting:
Question: whats 2 + 2

Answer: Let's think step by step.

> Finished chain.


' 2 + 2 = 4'

## Saving components separately
In the above example, we can see that the prompt and llm configuration information is saved in the same json as the overall chain. Alternatively, we can split them up and save them separately. This is often useful to make the saved components more modular. In order to do this, we just need to specify `llm_path` instead of the `llm` component, and `prompt_path` instead of the `prompt` component.

In [7]:
llm_chain.prompt.save("prompt.json")

In [8]:
!cat prompt.json

{
    "input_variables": [
        "question"
    ],
    "output_parser": null,
    "template": "Question: {question}\n\nAnswer: Let's think step by step.",
    "template_format": "f-string"
}

In [9]:
llm_chain.llm.save("llm.json")

In [10]:
!cat llm.json

{
    "model_name": "text-davinci-003",
    "temperature": 0.0,
    "max_tokens": 256,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "n": 1,
    "best_of": 1,
    "request_timeout": null,
    "logit_bias": {},
    "_type": "openai"
}

In [11]:
config = {
    "memory": None,
    "verbose": True,
    "prompt_path": "prompt.json",
    "llm_path": "llm.json",
    "output_key": "text",
    "_type": "llm_chain",
}
import json

with open("llm_chain_separate.json", "w") as f:
    json.dump(config, f, indent=2)

In [12]:
!cat llm_chain_separate.json

{
  "memory": null,
  "verbose": true,
  "prompt_path": "prompt.json",
  "llm_path": "llm.json",
  "output_key": "text",
  "_type": "llm_chain"
}

We can then load it in the same way

In [13]:
chain = load_chain("llm_chain_separate.json")

In [15]:
chain.run("whats 2 + 2")



> Entering new LLMChain chain...
Prompt after formatting:
Question: whats 2 + 2

Answer: Let's think step by step.

> Finished chain.


' 2 + 2 = 4'

# GigaChat summarization example

In [48]:
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.chat_models import GigaChat

giga = GigaChat(verify_ssl_certs=False, temperature=0.5)
summarize_chain = load_summarize_chain(giga, chain_type="map_reduce")

In [49]:
summarize_chain.save("summarize_chain.json")

In [51]:
!cat summarize_chain.json

{
    "memory": null,
    "verbose": false,
    "tags": null,
    "metadata": null,
    "input_key": "input_documents",
    "output_key": "output_text",
    "llm_chain": {
        "memory": null,
        "verbose": false,
        "tags": null,
        "metadata": null,
        "prompt": {
            "input_variables": [
                "text"
            ],
            "input_types": {},
            "output_parser": null,
            "partial_variables": {},
            "template": "Напиши краткое резюме следующего:\n\n\n\"{text}\"\n\n\nКРАТКОЕ РЕЗЮМЕ:",
            "template_format": "f-string",
            "validate_template": false,
            "_type": "prompt"
        },
        "llm": {
            "_type": "gigachat"
        },
        "output_key": "text",
        "output_parser": {
            "_type": "default"
        },
        "return_final_only": true,
        "llm_kwargs": {},
        "_type": "llm_chain"
    },
    "reduce_documents_chain": {
        "memory": null,
  

In [53]:
import os

from langchain.chains import load_chain

os.environ["GIGACHAT_USER"] = ...
os.environ["GIGACHAT_PASSWORD"] = ...
os.environ["GIGACHAT_VERIFY_SSL_CERTS"] = ...
os.environ["GIGACHAT_BASE_URL"] = ...

chain = load_chain("summarize_chain.json")

In [ ]:
!pip install wikipedia

In [58]:
from langchain.document_loaders import WikipediaLoader

big_article = WikipediaLoader(
    query="Великий аттрактор", lang="ru", load_max_docs=1, doc_content_chars_max=1000000
).load()[0]
docs = CharacterTextSplitter(chunk_size=5000, chunk_overlap=500).split_documents(
    [big_article]
)

In [59]:
res = chain.run(docs)

In [60]:
res

'"Великий аттрактор" — это гравитационная аномалия, находящаяся на расстоянии около 75 Мпк от Земли в созвездии Наугольник. Она представляет собой огромное сверхскопление галактик, масса которого составляет около 1015 M☉. Великий аттрактор влияет на движение галактик и их скоплений на участке пространства протяженностью в несколько сотен миллионов световых лет. Его существование подтверждается эффектами, которые он оказывает на движение галактик и их скоплений в этом районе Вселенной. Есть предположение, что Великий аттрактор может быть связан с другими источниками притяжения, скрытыми за плоскостью Млечного Пути.'